In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import confusion_matrix

### Data Pre-processing

In [26]:
df = pd.read_csv('train_test_small.csv')
df['DEP_BLOCK'] = pd.Categorical(df.DEP_BLOCK).codes
df['CARRIER_NAME'] = pd.Categorical(df.CARRIER_NAME).codes
df['DEPARTING_AIRPORT'] = pd.Categorical(df.DEPARTING_AIRPORT).codes
df['PREVIOUS_AIRPORT'] = pd.Categorical(df.DEP_BLOCK).codes

In [3]:
# df = df.drop(df.index[idx[:len(idx)//5*4]])
df.DEP_DEL15.value_counts()

0    414368
1     87682
Name: DEP_DEL15, dtype: int64

### Confusion Matrix : for viz

In [20]:
corr = df.corr()
# mask = np.zeros_like(corr, dtype=np.bool)
# mask[np.triu_indices_from(mask)] = True
# f, ax = plt.subplots(figsize=(25, 20))
# cmap = sns.diverging_palette(300, 20, as_cmap=True)


# sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
#             square=False, linewidths=0.1, cbar_kws={"shrink": .1}, ax = ax)

In [18]:
corr.to_parquet('df.parquet.gzip',compression='gzip')

In [21]:
def histogram(data):
    fig, axs = plt.subplots(5,5,figsize=(30, 30) )
    columns = list(df.columns[:2])+list(df.columns[3:])
    for i in range(5):
        for j in range(5):
                axs[i,j].hist(data[columns[i*5+j]])
                axs[i,j].set_title(columns[i*5+j])      

In [22]:
cols = ['DAY_OF_WEEK', 'DISTANCE_GROUP', 'DEP_BLOCK','SEGMENT_NUMBER', 'CONCURRENT_FLIGHTS', 'NUMBER_OF_SEATS',
       'CARRIER_NAME', 'AIRPORT_FLIGHTS_MONTH', 'AIRLINE_FLIGHTS_MONTH', 'AIRLINE_AIRPORT_FLIGHTS_MONTH', 
       'AVG_MONTHLY_PASS_AIRPORT', 'AVG_MONTHLY_PASS_AIRLINE', 'FLT_ATTENDANTS_PER_PASS', 'GROUND_SERV_PER_PASS',
       'PLANE_AGE', 'DEPARTING_AIRPORT', 'LATITUDE', 'PREVIOUS_AIRPORT', 'PRCP', 'SNOW', 'SNWD', 'AWND']
def scale(data, columns):
    for col in columns:
        data[col] = np.log( data[col] + 1 )
scale(df,cols)

In [24]:
# histogram(df)

### Leave one - searching the most important features

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

columns = list(df.columns[1:2])+list(df.columns[3:])
for column in columns:
    
#     X = df.drop(['DEP_DEL15', column], axis=1)
    X = df[[column, 'MONTH']]
    y = df['DEP_DEL15']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    idx = np.where(y_train == 0)[0]
    X_train = X_train.drop(X_train.index[idx[:len(idx)//5*4]])
    y_train = y_train.drop(y_train.index[idx[:len(idx)//5*4]])

    clf = xgb.XGBClassifier()
    clf.fit(X_train, y_train)
    
    print("\n\n\nWithout column: " + column)
    print("Train accuracy", clf.score(X_train, y_train))
    print("Test accuracy", clf.score(X_test, y_test))

### Models

In [ ]:
# Tried but useless
# from sklearn.preprocessing import StandardScaler, normalize
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test  = scaler.fit_transform(X_test)
# X_train = normalize(X_train, norm='l2')
# X_test  = normalize(X_test, norm='l2')

In [12]:
y_train.value_counts()

0    277512
1     58861
Name: DEP_DEL15, dtype: int64

In [9]:
X = df.drop('DEP_DEL15', axis=1)
y = df['DEP_DEL15']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42,shuffle=True)
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)
train_acc,t_acc = clf.score(X_train, y_train), clf.score(X_test, y_test)
print("Train accuracy",train_acc)
print("Test accuracy",t_acc)

[12:32:30] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Train accuracy 0.8450707993804497
Test accuracy 0.840291651828558


In [10]:
y_hat = clf.predict(X_test)
cm = confusion_matrix(y_test, y_hat)
print(cm /np.sum(cm,axis=0))

[[0.84676467 0.31830075]
 [0.15323533 0.68169925]]


In [ ]:
tr_acc = []
test_acc = []
X = df.drop('DEP_DEL15', axis=1)
y = df['DEP_DEL15']
X_tmp, X_test, y_tmp, y_test = train_test_split(X, y, test_size=0.20, random_state=42,shuffle=True)
for i in range(1,9):
    print("curr percentage",i/10)
    X_train,X_val,y_train,y_val = train_test_split(X_tmp, y_tmp, test_size=0.33, random_state=42,shuffle=True)
    idx = np.where(y_train == 0)[0]
    drop_per = int(len(idx)*i/10)
    X_train = X_train.drop(X_train.index[idx[:drop_per]])
    y_train = y_train.drop(y_train.index[idx[:drop_per]])

    clf = xgb.XGBClassifier()
    clf.fit(X_train, y_train)
    train_acc,t_acc = clf.score(X_train, y_train), clf.score(X_val, y_val)
    tr_acc.append(train_acc)
    test_acc.append(t_acc)
    print("Train accuracy",train_acc)
    print("Test accuracy",t_acc)
    y_hat = clf.predict(X_val)
    cm = confusion_matrix(y_val, y_hat)
    print(cm /np.sum(cm,axis=0))
    


In [ ]:
plt.plot(tr_acc,label="train")
plt.plot(test_acc,label="test")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Random forest
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
print("Train accuracy", clf.score(X_train, y_train))
print("Test accuracy", clf.score(X_test, y_test))

In [ ]:
from catboost import CatBoostClassifier
clf = CatBoostClassifier()
clf.fit(X_train, y_train, verbose=False)
print("Train accuracy", clf.score(X_train, y_train))
print("Test accuracy", clf.score(X_test, y_test))

In [ ]:
y_train_pred = clf.predict_proba(X_train)
y_test_pred = clf.predict_proba(X_test)

### Deep Learning

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
import time
import pickle
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping

#### AutoML

In [ ]:
LOG_DIR = f"{int(time.time())}"

# tensorboard = TensorBoard(log_dir=LOG_DIR)

def build_model(hp):  # random search passes this hyperparameter() object 
    model = keras.models.Sequential()

    model.add(Dense(hp.Int('input_units',min_value=50, max_value=200, step=50), 
                    input_shape=X_train.shape[1:],
                    activation='relu'))

    model.add( Dropout(hp.Float('dropout_rate',
                                    min_value=0.0,
                                    max_value=0.5,
                                    step=0.1) ) )
    
    for i in range(hp.Int('n_layers', 1, 2)):  # adding variation of layers.
        model.add(Dense(hp.Int(f'layer_{i}_units',
                                min_value=10,
                                max_value=150,
                                step=30), 
                                activation='relu'))
    
        model.add( Dropout(hp.Float(f'dropout_{i}_rate',
                                    min_value=0.0,
                                    max_value=0.5,
                                    step=0.1) ) )
    
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', 
              optimizer=tensorflow.keras.optimizers.Adam(),
              metrics =['accuracy'])
              
    return model
                  

tuner = RandomSearch(
    build_model,
    objective='accuracy',
    max_trials=10,  # how many variations on model?
    executions_per_trial=1) # how many trials per variation? (same model could perform differently)


tuner.search_space_summary()

tuner.search(x=X_train,
             y=y_train,
             epochs=5,
             batch_size=64)

tuner.results_summary()

#### DL model

In [ ]:
def build_model():  # random search passes this hyperparameter() object 
    model = keras.models.Sequential()

    model.add(Dense(200, input_shape=X_train.shape[1:], activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add( Dropout( 0.1 ) )
    model.add(Dense(70, activation='relu'))
    model.add( Dropout( 0.4 ) )
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', 
              optimizer=tensorflow.keras.optimizers.Adam(),
              metrics =['accuracy'])
              
    return model
                  

model = build_model()

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
reducer = tf.keras.callbacks.ReduceLROnPlateau( monitor='loss', factor=0.5, patience=3, verbose=1, mode='min')


model.fit(X_train,y_train, callbacks=[early_stopping, reducer], batch_size=256,verbose=1,epochs=1000 )


In [ ]:
import sklearn
from sklearn.metrics import accuracy_score 
y_hat = model.predict_classes(X_test)
# y_test
print("Test accuracy", sklearn.metrics.accuracy_score(y_test, y_hat))

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
y_hat = clf.predict(X_test)
cm = confusion_matrix(y_test, y_hat)
cm /np.sum(cm,axis=0)

### Precision Recall Curve


In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score
train_precision, train_recall, _ = precision_recall_curve(y_train, y_train_pred[:,1])
train_ap = average_precision_score(y_train, y_train_pred[:,1])

test_precision, test_recall, _ = precision_recall_curve(y_test, y_test_pred[:,1])
test_ap = average_precision_score(y_test, y_test_pred[:,1])


plt.figure(figsize=(20, 15))

axs = plt.subplot(2,2,1)
axs.set_title('Train Precision Recall Curve')
axs.set_xlabel('Recall')
axs.set_ylabel('Precission')
axs.plot(train_recall, train_precision, label = 'AP = {:.4f}'.format(train_ap))
axs.legend()

axs = plt.subplot(2,2,2)
axs.set_title('Test Precision Recall Curve')
axs.set_xlabel('Recall')
axs.set_ylabel('Precission')
axs.plot(test_recall, test_precision, label = 'AP = {:.4f}'.format(test_ap))
axs.legend()

### ROC curve

In [ ]:
from sklearn.metrics import roc_curve, auc
train_fpr, train_tpr, _ = roc_curve(y_train, y_train_pred[:,1])
train_auc = auc(train_fpr, train_tpr)

test_fpr, test_tpr, _ = roc_curve(y_test, y_test_pred[:,1])
test_auc = auc(test_fpr, test_tpr)

plt.figure(figsize=(20, 15))

axs = plt.subplot(2,2,1)
axs.set_title('Train ROC Curve')
axs.set_xlabel('False Positive Rate')
axs.set_ylabel('True Positive Rate')
axs.plot(train_fpr, train_tpr, label = 'AUC = {:.4f}'.format(train_auc))
axs.legend()

axs = plt.subplot(2,2,2)
axs.set_title('Test ROC Curve')
axs.set_xlabel('False Positive Rate')
axs.set_ylabel('True Positive Rate')
axs.plot(test_fpr, test_tpr, label = 'AUC = {:.4f}'.format(test_auc))
axs.legend()

In [ ]:
X_train